In [1]:
%load_ext sql

In [2]:
%sql sqlite:///./test.db

'Connected: @./test.db'

In [4]:
%%sql 
drop table if exists orderstatus;
create table OrderStatus
(
Quote_Id varchar(5),
Order_Id varchar(5),
Order_Status Varchar(20)
);

 * sqlite:///./test.db
Done.
Done.


[]

In [5]:
%%sql
Insert into OrderStatus Values ('A','A1','Delivered');
Insert into OrderStatus Values ('A','A2','Delivered');
Insert into OrderStatus Values ('A','A3','Delivered');
Insert into OrderStatus Values ('B','B1','Submitted');
Insert into OrderStatus Values ('B','B2','Delivered'); 
Insert into OrderStatus Values ('B','B3','Created');
Insert into OrderStatus Values ('C','C1','Submitted'); 
Insert into OrderStatus Values ('C','C2','Created');
Insert into OrderStatus Values ('C','C3','Submitted'); 
Insert into OrderStatus Values ('D','D1','Created');

 * sqlite:///./test.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [7]:
%sql select * from orderstatus

 * sqlite:///./test.db
Done.


Quote_Id,Order_Id,Order_Status
A,A1,Delivered
A,A2,Delivered
A,A3,Delivered
B,B1,Submitted
B,B2,Delivered
B,B3,Created
C,C1,Submitted
C,C2,Created
C,C3,Submitted
D,D1,Created


- When all Orders are in delivered status then Quote status should be ‘Complete’.
- When one or more Orders in delivered status then “ In Delivery”.
- When One or  more in Submitted status then “Awaiting for Submission” Else “Awaiting for Entry” by default

Note :- Order Priority should be Delivered, Submitted and Created
 If one order is in delivered and other one is in Submitted then Quote_Status should be “In Delivery” 

In [28]:
%%sql

select quote_id,
case when delivered=1 then 'Complete'
when indelivery = 1 then 'In Delivery'
when awaitingsubmission = 1 then 'Awaiting for Submission'
else 'Awaiting for Entry' end status from ( 
select quote_id,
max(delivered) delivered,
max(indelivery) indelivery,
max(awaitingsubmission) awaitingsubmission
from (
select o.quote_id,o.order_status,cntorderstatus,
max(case when order_status='Delivered' and cntorderstatus=1 then 1 else 0 end) Delivered,
max(case when order_status='Delivered' then 1 else 0 end) InDelivery,
max(case when order_status='Submitted' then 1 else 0 end) AwaitingSubmission
from 
(select distinct quote_id,order_status from orderstatus) o
join 
(select quote_id,count(distinct(order_status)) cntorderstatus from orderstatus
group by quote_id) od
on o.quote_id = od.quote_id
group by o.quote_id,o.order_status
)x
group by quote_id)y

 * sqlite:///./test.db
Done.


quote_id,status
A,Complete
B,In Delivery
C,Awaiting for Submission
D,Awaiting for Entry
